#  Detect computer security breach using RNNs and LSTMs

#1. Data Processing: This data set is a bit messy, so the preprocessing portion is largely a tutorial to make sure students have data ready for keras. 

a) Import the following libraries: 

In [67]:
!pip install keras
!pip install tensorflow

b) We will read the code in slightly differently than before: 

In [68]:
import sys
import os
import json
import pandas

import numpy
import optparse

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from keras.layers import Embedding  
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from collections import OrderedDict


c) We then need to convert to a numpy.ndarray type: 

d) Check the shape of the data set - it should be (26773, 2). Spend some time looking at the data.

e) Store all rows and the 0th index as the feature data: 

f) Store all rows and index 1 as the target variable:

In [69]:
dataframe = pandas.read_csv("dev-access.csv", engine='python', quotechar='|', header=None)

In [70]:
dataset = dataframe.values

In [71]:
print("Shape of dataset:", dataset.shape)
print("First 5 rows:")
for row in dataset[:5]:
    print("Row:")
    for column_idx, column_value in enumerate(row):
        print(f"Column {column_idx}: {column_value}")


Shape of dataset: (26773, 2)
First 5 rows:
Row:
Column 0: {"timestamp":1502738402847,"method":"post","query":{},"path":"/login","statusCode":401,"source":{"remoteAddress":"88.141.113.237","referer":"http://localhost:8002/enter"},"route":"/login","headers":{"host":"localhost:8002","accept-language":"en-us","accept-encoding":"gzip, deflate","connection":"keep-alive","accept":"*/*","referer":"http://localhost:8002/enter","cache-control":"no-cache","x-requested-with":"XMLHttpRequest","content-type":"application/json","content-length":"36"},"requestPayload":{"username":"Carl2","password":"bo"},"responsePayload":{"statusCode":401,"error":"Unauthorized","message":"Invalid Login"}}
Column 1: 0
Row:
Column 0: {"timestamp":1502738402849,"method":"post","query":{},"path":"/login","statusCode":401,"source":{"remoteAddress":"88.141.113.237"},"route":"/login","headers":{"host":"localhost:8002","connection":"keep-alive","cache-control":"no-cache","accept":"*/*","accept-encoding":"gzip, deflate, br","

In [74]:
X = dataset[:,0]
Y = dataset[:,1]

g) In the next step, we will clean up the predictors. This includes removing features that are not valuable, such as timestamp and source. 


In [52]:
for index, item in enumerate(X):
    # Quick hack to space out json elements
    reqJson = json.loads(item, object_pairs_hook=OrderedDict)
    del reqJson['timestamp']
    del reqJson['headers']
    del reqJson['source']
    del reqJson['route']
    del reqJson['responsePayload']
    X[index] = json.dumps(reqJson, separators=(',', ':'))

h) We next will tokenize our data, which just means vectorizing our text. Given the data we will tokenize every character (thus char_level = True)

In [76]:
tokenizer = Tokenizer(filters='\t\n', char_level=True)
tokenizer.fit_on_texts(X)

# we will need this later
num_words = len(tokenizer.word_index)+1
X = tokenizer.texts_to_sequences(X)


i) Need to pad our data as each observation has a different length

In [77]:
max_log_length = 1024
X_processed = sequence.pad_sequences(X, maxlen=max_log_length)

j) Create your train set to be 75% of the data and your test set to be 25%

In [85]:
from sklearn.model_selection import train_test_split

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, Y, test_size=0.25, random_state=42)

# Printing the shapes of the train and test sets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (20079, 1024)
Shape of X_test: (6694, 1024)
Shape of y_train: (20079,)
Shape of y_test: (6694,)


2. Model 1 - RNN: The first model will be a pretty minimal RNN with only an embedding layer, simple RNN and Dense layer. The next model we will add a few more layers. 

In [57]:
##adding embedding layer

In [107]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense

# Creating a new instance of a Sequential model
model = Sequential()

# Adding an Embedding layer
model.add(Embedding(input_dim=num_words, output_dim=32, input_length=max_log_length))

# Adding a SimpleRNN layer
model.add(SimpleRNN(units=32, activation='relu'))

# Adding a Dense layer
model.add(Dense(units=1, activation='sigmoid'))

# Building the model 
model.build(input_shape=(num_words, max_log_length))

# Compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Printing the model summary
print(model.summary())


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_13 (Embedding)        │ (None, 1024, 32)       │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_9 (SimpleRNN)        │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,161 (16.25 KB)

 Trainable params: 4,161 (16.25 KB)

 Non-trainable params: 0 (0.00 B)

None


In [108]:
# Compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [109]:
# Printing the model summary
print(model.summary())


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_13 (Embedding)        │ (None, 1024, 32)       │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_9 (SimpleRNN)        │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,161 (16.25 KB)

 Trainable params: 4,161 (16.25 KB)

 Non-trainable params: 0 (0.00 B)

None


In [111]:
import numpy as np

# Converting y_train to integer data type
y_train = y_train.astype(np.int32)


In [112]:
# Fitting the model on the training data
history = model.fit(X_train, y_train, epochs=3, batch_size=128, validation_split=0.25)


Epoch 1/3
118/118 ━━━━━━━━━━━━━━━━━━━━ 15s 111ms/step - accuracy: 0.7048 - loss: 0.5508 - val_accuracy: 0.7532 - val_loss: 0.4341
Epoch 2/3
118/118 ━━━━━━━━━━━━━━━━━━━━ 13s 110ms/step - accuracy: 0.7456 - loss: 0.4415 - val_accuracy: 0.7532 - val_loss: 0.4339
Epoch 3/3
118/118 ━━━━━━━━━━━━━━━━━━━━ 13s 112ms/step - accuracy: 0.7491 - loss: 0.4386 - val_accuracy: 0.7532 - val_loss: 0.4328


In [113]:
import numpy as np

# Converting y_test to integers
y_test = y_test.astype(int)


In [114]:
# Using the .evaluate() method to get the loss value and accuracy value on the test data
loss, accuracy = model.evaluate(X_test, y_test, batch_size=128)

# Printing the loss value and accuracy value
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7416 - loss: 0.4340
Test Loss: 0.43643516302108765
Test Accuracy: 0.740962028503418


In [ ]:
###3) Model 2 - LSTM + Dropout Layers:

In [143]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dropout, Dense

# Creating a new instance of a Sequential model
model_lstm = Sequential()

# Adding an Embedding layer
model_lstm.add(Embedding(input_dim=num_words, output_dim=32, input_length=max_log_length))

# Adding an LSTM layer with recurrent dropout
model_lstm.add(LSTM(units=64, recurrent_dropout=0.5))

# Adding a Dropout layer
model_lstm.add(Dropout(0.5))

# Adding a Dense layer
model_lstm.add(Dense(units=1, activation='sigmoid'))

# Building the model
model_lstm.build(input_shape=(num_words, max_log_length))



In [144]:
# Compiling the model
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Printing the model summary
print(model_lstm.summary())

# Fitting the model on the training data
history_lstm = model_lstm.fit(X_train, y_train, epochs=3, batch_size=128, validation_split=0.25)

# Evaluating the model on test data
loss_lstm, accuracy_lstm = model_lstm.evaluate(X_test, y_test, batch_size=128)

# Printing the loss value and accuracy value
print("Test Loss:", loss_lstm)
print("Test Accuracy:", accuracy_lstm)


Model: "sequential_31"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_30 (Embedding)        │ (64, 1024, 32)         │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_23 (LSTM)                  │ (64, 64)               │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (64, 64)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ (64, 1)                │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26,945 (105.25 KB)

 Trainable params: 26,945 (105.25 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/3
118/118 ━━━━━━━━━━━━━━━━━━━━ 86s 708ms/step - accuracy: 0.6450 - loss: 0.6119 - val_accuracy: 0.7532 - val_loss: 0.4441
Epoch 2/3
118/118 ━━━━━━━━━━━━━━━━━━━━ 83s 702ms/step - accuracy: 0.7395 - loss: 0.4591 - val_accuracy: 0.7532 - val_loss: 0.4378
Epoch 3/3
118/118 ━━━━━━━━━━━━━━━━━━━━ 79s 670ms/step - accuracy: 0.7391 - loss: 0.4470 - val_accuracy: 0.7532 - val_loss: 0.4330
53/53 ━━━━━━━━━━━━━━━━━━━━ 7s 136ms/step - accuracy: 0.7416 - loss: 0.4337
Test Loss: 0.436225950717926
Test Accuracy: 0.740962028503418


##4) Recurrent Neural Net Model 3: Build Your Own

In [145]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout

# Defining the model
model = Sequential()

model.add(Embedding(num_words, 128, input_length=max_log_length))  # Adjust parameters as needed

# Stacking 2 LSTM layers
model.add(LSTM(64, return_sequences=True))  # 64 units, return sequences for next LSTM
model.add(Dropout(0.2))  # Add dropout for regularization

model.add(LSTM(32))  # 32 units for final layer

# New Layer: Dense layer with ReLU activation 
model.add(Dense(128, activation='relu'))

# Output layer
model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

# Calling build (optional in this case)
model.build(input_shape=(num_words, max_log_length))  # Provide input shape if desired

In [146]:
# Compiling the model with a new optimizer (e.g., RMSprop)
model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

# Printing  the model summary
model.summary()

# Training the model with validation split
model.fit(X_train, y_train, epochs=3, batch_size=128, validation_split=0.25)

# Evaluating the model on test data
test_loss, test_acc = model.evaluate(X_test, y_test, batch_size=128)

# Printing test loss and accuracy
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)


Model: "sequential_32"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_31 (Embedding)        │ (64, 1024, 128)        │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_24 (LSTM)                  │ (64, 1024, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (64, 1024, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_25 (LSTM)                  │ (64, 32)               │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_58 (Dense)                │ (64, 128)              │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_59 (Dense)                │ (64, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 74,369 (290.50 KB)

 Trainable params: 74,369 (290.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
118/118 ━━━━━━━━━━━━━━━━━━━━ 284s 2s/step - accuracy: 0.6971 - loss: 0.5710 - val_accuracy: 0.7532 - val_loss: 0.4392
Epoch 2/3
118/118 ━━━━━━━━━━━━━━━━━━━━ 282s 2s/step - accuracy: 0.7477 - loss: 0.4425 - val_accuracy: 0.7532 - val_loss: 0.4341
Epoch 3/3
118/118 ━━━━━━━━━━━━━━━━━━━━ 283s 2s/step - accuracy: 0.7434 - loss: 0.4436 - val_accuracy: 0.7532 - val_loss: 0.4320
53/53 ━━━━━━━━━━━━━━━━━━━━ 21s 394ms/step - accuracy: 0.7416 - loss: 0.4331
Test Loss: 0.4355980157852173
Test Accuracy: 0.740962028503418


Conceptual Questions: 

5) Explain the difference between the relu activation function and the sigmoid activation function.

Ans: Both ReLU (Rectified Linear Unit) and sigmoid are popular activation functions used in artificial neural networks.
Sigmoid squashes any real number between positive and negative infinity into a value between 0 and 1. It produces an S-shaped curve.The output values range continuously from 0 to 1.
ReLU sets any negative input value to zero and keeps positive input values unchanged. It essentially acts like a threshold function.The output can be either 0 ( fo negative inputs)  or 1 (for poitive inputs). 
 
6) Describe what one epoch actually is (epoch was a parameter used in the .fit() method).

Ans: In machine learning, an epoch represents a single, complete cycle of training our model on the entire dataset. It's like a student going through their entire textbook (training data) once. During this epoch, each data point is fed through the model, predictions are made, and errors are calculated. The model then uses this error information to adjust its internal parameters.  Multiple epochs allow the model to learn progressively from the data.

7) Explain how dropout works (you can look at the keras code and/or documentation) for (a) training, and (b) test data sets.

Ans: Dropout is a regularization technique to prevent neural networks from overfitting during training.In out code we used dropout rate of 0.2. During training, 20% (0.2 dropout rate) of the activations in the previous LSTM layer  are randomly chosen to be set to zero.Dropout discourages the network from relying too heavily on specific neurons. This helps prevent overfitting and improves the model's ability to generalize to unseen data.
The dropout layer acts like a training switch to prevent our RNN from overfitting on login data. During training, it randomly removes 20% of the analysts (activations from the previous layer) working on each login attempt. This forces the remaining analysts to collaborate effectively and learn to solve the problem without relying on specific colleagues. To compensate for the missing analysts, the workload of the remaining ones is slightly increased. This discourages the network from becoming overly reliant on any particular neuron and helps it generalize better to unseen data. However, during testing on unseen login attempts, the dropout switch is turned off. All analysts work together, and their contributions are used directly without any adjustments, ensuring the most accurate predictions possible.


8) Explain why problems such as this homework assignment are better modeled with RNNs than CNNs. What type of problem will CNNs outperform RNNs on?

Ans: RNNs excel at handling sequential information like login attempts, allowing them to capture temporal relationships (e.g., frequent attempts from a new location).RNNs can efficiently process these sequences of different lengths.
CNNs are powerful for analyzing data arranged in grids, like images or structured time series.
CNNs focus on capturing local patterns within the data, making them suitable for tasks where features are localized (e.g., identifying objects in images).

9) Explain what RNN problem is solved using LSTM and briefly describe how.

Ans : LSTMs overcome the vanishing gradient problem by selectively controlling the flow of information and maintaining long-term dependencies within their cell state.Speech Recognition,Machine Translation,Time Series Forecasting,Video Analysis.STMs introduce a special gating mechanism that controls the flow of information within the network. This mechanism allows LSTMs to:Select Relevant Information: LSTMs use gates (forget gate, input gate, and output gate) to selectively remember or forget information from previous steps. The forget gate decides what information to discard from the cell state (internal memory), the input gate controls what new information to store, and the output gate determines what information from the cell state to use in the current output.
Maintain Long-Term Dependencies: The cell state in LSTMs acts like a memory buffer that can store information for extended periods. Unlike the hidden state in a regular RNN, the cell state's information is not directly overwritten but selectively updated through the gating mechanism. This allows LSTMs to learn and retain information from distant parts of the sequence.

In [167]:
import pandas as pd
import numpy as np
from typing import Tuple
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

def create_data_for_NN(
    data: pd.DataFrame, Y_var: str, lag: int, test_ratio: float
) -> Tuple[np.array, np.array, np.array, np.array]:
  """Function to return lagged time series data after train-test split

  Args:
      data (pd.DataFrame): Raw time series data frame
      Y_var (str): String with the name of y variable
      lag (int): number of lagged records to consider
      test_ratio (float): ratio of data to consider for test set

  Returns:
      Tuple[np.array, np.array, np.array, np.array]: Lagged and split numpy arrays
  """
  y = data[Y_var].tolist()

  X, Y = [], []

  if len(y) - lag <= 0:
    X.append(y)
  else:
    for i in range(len(y) - lag):
      Y.append(y[i + lag])
      X.append(y[i: (i + lag)])

  X, Y = np.array(X), np.array(Y)

  # Reshaping the X array to an LSTM input shape
  X = np.reshape(X, (X.shape[0], X.shape[1], 1))

  # Creating training and test sets
  X_train = X
  X_test = []

  Y_train = Y
  Y_test = []

  if test_ratio > 0:
    index = round(len(X) * test_ratio)
    X_train = X[: (len(X) - index)]
    X_test = X[-index:]

    Y_train = Y[: (len(X) - index)]
    Y_test = Y[-index:]

  return X_train, X_test, Y_train, Y_test




In [177]:
# Loading data
data = pd.read_csv("DAYTON_hourly.csv")

# Checking for missing values
print("Missing values:")
print(data.isnull().sum())


# Converting o datetime format
data["Datetime"] = pd.to_datetime(data["Datetime"])

# Sorting Datetime for a time series order
data.sort_values(by="Datetime", inplace=True)

# Defining target variable
target_variable = "DAYTON_MW"

# Defining values to test (3 hours and 24 hours)
lag_values = [3, 24]

# Looping thru different lag values to create data for various models
for lag in lag_values:
  # Split data into training and testing sets with a 15% test ratio
  X_train, X_test, Y_train, Y_test = create_data_for_NN(data, target_variable, lag, 0.15)

    
  # Defining and fitting a single layer LTSM model
  model = Sequential()
  model.add(LSTM(50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
  model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")



  # Adding early stopping to prevent overfitting
  early_stopping = EarlyStopping(monitor="val_loss", patience=5)
  history = model.fit(
      X_train,
      Y_train,
      epochs=20,
      validation_data=(X_test, Y_test),
      callbacks=[early_stopping],
  )

  # Making predictions on test data
  y_predicted = model.predict(X_test)

  # Calculating RMSE on test data
  rmse = mean_squared_error(Y_test, y_predicted, squared=False)
  print(f"Single LSTM with lag {lag} hours - RMSE: {rmse:.4f}")

# Printing single LSTM model summary for this lag
print(f"Single LSTM Model Summary (lag {lag} hours):")
print(model.summary())  # Added printing the model summary
  

Missing values:
Datetime     0
DAYTON_MW    0
dtype: int64
Epoch 1/20


C:\Apps\Anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3222/3222 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 4299803.5000 - val_loss: 4185320.7500
Epoch 2/20
3222/3222 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 4208990.5000 - val_loss: 4104090.2500
Epoch 3/20
3222/3222 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 4135594.7500 - val_loss: 4024422.7500
Epoch 4/20
3222/3222 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 4060600.0000 - val_loss: 3945608.7500
Epoch 5/20
3222/3222 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 3974140.7500 - val_loss: 3867569.7500
Epoch 6/20
3222/3222 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 3897182.0000 - val_loss: 3790339.0000
Epoch 7/20
3222/3222 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 3818366.7500 - val_loss: 3713962.5000
Epoch 8/20
3222/3222 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 3737175.0000 - val_loss: 3638369.2500
Epoch 9/20
3222/3222 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 3663213.5000 - val_loss: 3563589.2500
Epoch 10/20
3222/3222 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 3591011.7500 - val_loss: 3489666.2500
Epoch 11/2

C:\Apps\Anaconda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3221/3221 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 4283026.5000 - val_loss: 4077246.7500
Epoch 2/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 4084842.2500 - val_loss: 3891369.7500
Epoch 3/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 3890791.5000 - val_loss: 3712723.5000
Epoch 4/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 3717820.0000 - val_loss: 3539183.2500
Epoch 5/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 3546488.2500 - val_loss: 3370045.5000
Epoch 6/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 3372276.0000 - val_loss: 3205176.7500
Epoch 7/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 3200934.2500 - val_loss: 3044704.0000
Epoch 8/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 3047989.0000 - val_loss: 2888685.5000
Epoch 9/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 2889904.0000 - val_loss: 2736957.0000
Epoch 10/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 2736683.0000 - val_loss: 2589598.7500


...

 Total params: 31,355 (122.48 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 20,904 (81.66 KB)

None


Fit a bidirectional LSTM with lag = 24 hours

In [171]:
import pandas as pd
import numpy as np
from typing import Tuple
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

def create_data_for_NN(
    data: pd.DataFrame, Y_var: str, lag: int, test_ratio: float
) -> Tuple[np.array, np.array, np.array, np.array]:
  """Function to return lagged time series data after train-test split

  Args:
      data (pd.DataFrame): Raw time series data frame
      Y_var (str): String with the name of y variable
      lag (int): number of lagged records to consider
      test_ratio (float): ratio of data to consider for test set

  Returns:
      Tuple[np.array, np.array, np.array, np.array]: Lagged and split numpy arrays
  """
  y = data[Y_var].tolist()

  X, Y = [], []

  if len(y) - lag <= 0:
    X.append(y)
  else:
    for i in range(len(y) - lag):
      Y.append(y[i + lag])
      X.append(y[i: (i + lag)])

  X, Y = np.array(X), np.array(Y)

  # Reshaping the X array to an LSTM input shape
  X = np.reshape(X, (X.shape[0], X.shape[1], 1))

  # Creating training and test sets
  X_train = X
  X_test = []

  Y_train = Y
  Y_test = []

  if test_ratio > 0:
    index = round(len(X) * test_ratio)
    X_train = X[: (len(X) - index)]
    X_test = X[-index:]

    Y_train = Y[: (len(X) - index)]
    Y_test = Y[-index:]

  return X_train, X_test, Y_train, Y_test


# Loading data
data = pd.read_csv("DAYTON_hourly.csv")

# Checking for missing values
print("Missing values:")
print(data.isnull().sum())



# Converting Datetime to datetime format
data["Datetime"] = pd.to_datetime(data["Datetime"])

# Sorting by Datetime for a time series order
data.sort_values(by="Datetime", inplace=True)

# Defining target variable
target_variable = "DAYTON_MW"

# Defining lag values to test
lag_values = [3, 24]

# Loop through different lag values to create data for various models
for lag in lag_values:
  # Split data into training and testing sets with a 15% test ratio
  X_train, X_test, Y_train, Y_test = create_data_for_NN(data, target_variable, lag, 0.15)



  # Defining and fitting a bidirectional LSTM model with lag 24 hours
  if lag == 24:
    inputs = Input(shape=(X_train.shape[1], 1))  # No need for input_tensor argument
    model = Sequential()
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))  # Second LSTM layer in the bidirectional architecture
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")


    # Adding early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor="val_loss", patience=5)
    history = model.fit(
        X_train,
        Y_train,
        epochs=20,
        validation_data=(X_test, Y_test),
        callbacks=[early_stopping],
    )


Missing values:
Datetime     0
DAYTON_MW    0
dtype: int64
Epoch 1/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 27s 8ms/step - loss: 4114158.0000 - val_loss: 3589439.2500
Epoch 2/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - loss: 3489964.2500 - val_loss: 3021333.2500
Epoch 3/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - loss: 2933061.0000 - val_loss: 2506704.0000
Epoch 4/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - loss: 2426050.7500 - val_loss: 2044606.8750
Epoch 5/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 26s 8ms/step - loss: 1974631.0000 - val_loss: 1634408.2500
Epoch 6/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - loss: 1575862.5000 - val_loss: 1276008.2500
Epoch 7/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 26s 8ms/step - loss: 1226145.5000 - val_loss: 968679.6875
Epoch 8/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 26s 8ms/step - loss: 925594.7500 - val_loss: 712503.7500
Epoch 9/20
3221/3221 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - loss: 678807.6250 - val_loss: 506620.0625
Epoch 10/20
3221/3221 ━━━━━━━━━━━━━━━

In [173]:
# Printing the bidirectional LSTM model summary
print("Bidirectional LSTM Model Summary:")
print(model.summary())
print ()


Bidirectional LSTM Model Summary:


Model: "sequential_54"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_48 (LSTM)                  │ (None, 24, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_49 (LSTM)                  │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 91,955 (359.20 KB)

 Trainable params: 30,651 (119.73 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 61,304 (239.47 KB)

None


# THE END